<a href="https://colab.research.google.com/github/luasampaio/CienciaDados/blob/main/54_UDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


- @author: Luciana Sampaio:
- @date : 05/09/25

## Descrição geral:
Objetivo: encapsular uma lógica de transformação ou cálculo que não existe como função nativa.

Uso: pode ser chamada dentro de queries SQL ou em DataFrames, como se fosse uma função interna.

Contexto: muito comum em ambientes de Big Data (ex.: Databricks, Spark), quando é necessário aplicar regras de negócio específicas, manipulação de strings, cálculos customizados, criptografia, mascaramento de dados, etc.

Importando Bibliotecas

In [9]:
import time
import random
import string

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, pandas_udf, col, lit
from pyspark.sql.types import StringType
from pyspark.sql.functions import initcap  # função nativa p/ baseline

In [19]:
# Configuração do Spark
# ----------------------------
spark = (SparkSession.builder
         .appName("UDF vs PandasUDF Benchmark")
         # aumente/exclua estas configs conforme seu cluster
         .config("spark.sql.shuffle.partitions", "200")
         .getOrCreate())

# MUITO importante: habilitar Arrow (acelera Pandas UDF)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [20]:
# Definir a UDF em Python
def get_full_name(first_name, last_name):
    if first_name is None:
        first_name = ""
    if last_name is None:
        last_name = ""
    return (first_name + " " + last_name).strip()

In [21]:
# Registrar como UDF no Spark
get_full_name_udf = udf(get_full_name, StringType())

In [22]:
# Exemplo de DataFrame
data = [("Luciana", "Sampaio"), ("Jose", "Souza"), ("Angelica", None)]
df = spark.createDataFrame(data, ["first_name", "last_name"])

In [23]:
# Usar a UDF
df = df.withColumn("full_name", get_full_name_udf("first_name", "last_name"))

In [24]:
df.show(truncate=False)

+----------+---------+---------------+
|first_name|last_name|full_name      |
+----------+---------+---------------+
|Luciana   |Sampaio  |Luciana Sampaio|
|Jose      |Souza    |Jose Souza     |
|Angelica  |NULL     |Angelica       |
+----------+---------+---------------+



In [33]:
import pandas as pd
from pyspark.sql.functions import pandas_udf

@pandas_udf(StringType())
def pandas_capitalize_udf(names: pd.Series) -> pd.Series:

# Use efficient, vectorized pandas operations
  return names.str.capitalize()

In [26]:
# Aplicando a Pandas UDF
df = df.withColumn("name_cap", pandas_capitalize_udf("first_name"))

In [27]:
df.show()

+----------+---------+---------------+--------+
|first_name|last_name|      full_name|name_cap|
+----------+---------+---------------+--------+
|   Luciana|  Sampaio|Luciana Sampaio| Luciana|
|      Jose|    Souza|     Jose Souza|    Jose|
|  Angelica|     NULL|       Angelica|Angelica|
+----------+---------+---------------+--------+



In [28]:
df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- name_cap: string (nullable = true)

